In [71]:
import os 
import json
import pandas as pd 
import traceback

In [72]:
from langchain.chat_models import ChatOpenAI

In [73]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env

True

In [74]:
KEY=os.getenv("OPENAI_API_KEY")

In [75]:
# OpenAI API  
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-4-turbo", temperature=0.5)

In [76]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x171503520>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1698e6e30>, model_name='gpt-4-turbo', temperature=0.5, openai_api_key='sk-proj-xELrJKo2AbdEd3shVInST3BlbkFJE6IaDTpTiKM4IVMFSp34', openai_proxy='')

In [77]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [78]:
# Response Format 
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    
    "2": {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    
    "3": {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    
    "4": {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    
    "5": {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    }
}

In [79]:
# Prompt Template 
TEMPLATE=""""
Text:{text}
You are expert MCQ maker. Given the above text, it is your job to \ create a quiz of 
{number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the text as well. 
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \ 
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [80]:
# Define promptTemplate 
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number", "subject","tone","response_json"],
    template=TEMPLATE
)

In [81]:
quiz_chain =LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [82]:
# Evaluate the quiz
TEMPLATE2="""
You are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity 
if the quiz is not at per with the cognitive and analytics abilities of the students,\
update the quiz questions which needs to be changed the tone such that it perfectly fits the students
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [83]:
quiz_evaluate_prompt = PromptTemplate(input_variables=["subject", " quiz"], template=TEMPLATE2)

In [84]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluate_prompt, output_key="review", verbose=True)

In [85]:
# Combine the chains 
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number", "subject","tone","response_json", ],
                                          output_variables=["quiz", "review"], verbose=True)

In [86]:
file_path = "/Users/abdoulabdillahi/Desktop/Generative AI /Generative_AI_Project/mcqgen/data.txt"

In [87]:
file_path

'/Users/abdoulabdillahi/Desktop/Generative AI /Generative_AI_Project/mcqgen/data.txt'

In [88]:
# Read data 
with open(file_path, 'r') as file:
    TEXT = file.read()

In [89]:
print(TEXT)

Artificial intelligence

Machine learning as subfield of AI[22]
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[25] Work on symbolic/knowledge-ba

In [90]:
# Serialize the Python dictionary into a JSON-formatted string 
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [91]:
NUMBER = 5
SUBJECT = "machine learning"
TONE="simple"

The get_openai_callbacks() functions goes over how to track your token usage for specific calls. It is currently only implemented for the OpenAI API.

In [92]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number":NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"
Text:Artificial intelligence

Machine learning as subfield of AI[22]
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]: 488  By 198

In [93]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total cost: {cb.total_cost}")

Total Tokens: 3107
Prompt Tokens: 2354
Completion Tokens: 753
Total cost: 0.046130000000000004


In [94]:
response

{'text': 'Artificial intelligence\n\nMachine learning as subfield of AI[22]\nAs a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]:\u200a488\u200a\n\nHowever, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]:\u200a488\u200a By 1980, expert systems had come to dominate AI, and statistics was out of favor

In [95]:
quiz = response.get("quiz")

In [96]:
quiz = json.loads(quiz)

In [97]:
# Create df 
quiz_table_data = []

for key, value in quiz.items():
    mcq=value["mcq"]
    option=" | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "choices": option,"correct":correct})

In [98]:
quiz_table_data

[{'MCQ': 'What caused the rift between AI and machine learning in the early days?',
  'choices': 'a:The focus on probabilistic systems | b:The emphasis on logical, knowledge-based approaches | c:The invention of neural networks | d:The use of inductive logic programming',
  'correct': 'b'},
 {'MCQ': 'Which of the following is NOT a focus of machine learning as it developed as its own field in the 1990s?',
  'choices': 'a:Achieving artificial intelligence | b:Tackling solvable practical problems | c:Borrowing methods from statistics and probability theory | d:Using fuzzy logic',
  'correct': 'a'},
 {'MCQ': 'What is the relationship between machine learning and data compression?',
  'choices': 'a:Machine learning algorithms cannot compress data | b:Data compression algorithms can be used for prediction in machine learning | c:Machine learning has no impact on improving data compression techniques | d:Machine learning exclusively uses lossy compression methods',
  'correct': 'b'},
 {'MCQ'

In [99]:
# Convert df 
quiz = pd.DataFrame(quiz_table_data)

In [100]:
quiz.to_csv("machinelearning.csv", index=False)

In [2]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_26_2024_19_57_36'